In [1]:
import pandas as pd
from sklearn.impute import KNNImputer
import numpy as np
from sklearn.linear_model import LinearRegression

In [2]:
df = pd.read_csv('data/final_scrapped_immobiliers.csv')

# Les biens trop chers sont evidemment à vendre
df.loc[(df['Nature'].isnull()) & (df['Prix'] > 100000), 'Nature'] = 'À Vendre'

# Check for missing values
print("Missing values per column:")
print(df.isnull().sum())

Missing values per column:
Région                     0
Localisation            2070
Type de bien            1319
Nature                    31
Superficie               335
Salles de bains          315
Chambres                 221
Prix                      15
Date_De_Modification    2240
Annonce                    0
dtype: int64


Handling missing values in 'Type de bien'

In [3]:
# Convert 'Chambres' to numeric
df['Chambres'] = pd.to_numeric(df['Chambres'], errors='coerce')

# Fill missing values in 'Type de bien'
df.loc[(df['Type de bien'].isnull()) & 
       (df['Chambres'].between(1, 4)) & 
       (df['Salles de bains'].between(1, 2)), 'Type de bien'] = 'Appartement'

df.loc[(df['Type de bien'].isnull()) & 
    (df['Chambres'] > 4) & 
    (df['Salles de bains'] >= 2), 'Type de bien'] = 'Villa'

# Drop the rows that has no 'Type de bien' value
df = df.dropna(subset=['Type de bien'])

# Check missing values after the update
print("Missing values in 'Type de bien' after:", df['Type de bien'].isnull().sum())

Missing values in 'Type de bien' after: 0


Handling missing values in 'Prix' with a linear regression model

In [5]:
# Fill rows where 'Prix' is not NaN or 0
X = df.loc[(df['Prix'].notnull()) & (df['Prix'] != 0), ['Superficie', 'Chambres', 'Salles de bains', 'Type de bien', 'Localisation']]
X = pd.get_dummies(X, columns=['Type de bien', 'Localisation'], drop_first=True)
y = df.loc[(df['Prix'].notnull()) & (df['Prix'] != 0), 'Prix']

regressor = LinearRegression()
regressor.fit(X, y)

X_missing_prix = df.loc[df['Prix'].isnull() | (df['Prix'] == 0), ['Superficie', 'Chambres', 'Salles de bains', 'Type de bien', 'Localisation']]
X_missing_prix = pd.get_dummies(X_missing_prix, columns=['Type de bien', 'Localisation'], drop_first=True)
X_missing_prix = X_missing_prix.reindex(columns=X.columns, fill_value=0)
X_missing_prix.fillna({
    'Superficie': df['Superficie'].median(), 
    'Chambres': df['Chambres'].median(), 
    'Salles de bains': df['Salles de bains'].mode().iloc[0]
}, inplace=True)

predicted_prix = regressor.predict(X_missing_prix)
df.loc[df['Prix'].isnull() | (df['Prix'] == 0), 'Prix'] = predicted_prix
# Remove ouliers based on visualizations
df['Prix'] = df['Prix'].clip(upper=df['Prix'].quantile(0.99))

df.drop(columns=['Annonce'], inplace=True)
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)

In [6]:
df['Prix'] = df['Prix'].clip(upper=df['Prix'].quantile(0.99))

In [4]:
# Drop 'Date_De_Modification'
df.drop(columns=['Date_De_Modification'], inplace=True, errors='ignore')

# Handle Missing Values for 'Localisation' using KNN
df['Localisation_num'] = df['Localisation'].astype('category').cat.codes
localisation_imputer = KNNImputer(n_neighbors=5)
df['Localisation_num'] = localisation_imputer.fit_transform(df[['Localisation_num']])

localisation_mapping = dict(enumerate(df['Localisation'].astype('category').cat.categories))
df['Localisation'] = df['Localisation_num'].round().astype(int).map(localisation_mapping)
df.drop(columns=['Localisation_num'], inplace=True)

# Handle Missing Values for 'Superficie' by filling based on 'Type de bien'
df['Superficie'] = df.groupby('Type de bien')['Superficie'].transform(lambda x: x.fillna(x.median()))

# Handle Missing Values for 'Chambres' and make them integer
df['Chambres'] = pd.to_numeric(df['Chambres'], errors='coerce')  # Ensure numeric values
knn_imputer = KNNImputer(n_neighbors=5)
df[['Chambres']] = knn_imputer.fit_transform(df[['Chambres']])
df['Chambres'] = df['Chambres'].round()
df['Chambres'] = df['Chambres'].replace(0, np.nan)
df['Chambres'] = df.groupby('Type de bien')['Chambres'].transform(lambda x: x.fillna(x.median()))

df['Salles de bains'] = df.groupby(['Type de bien', 'Localisation'])['Salles de bains'].transform(
    lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x.fillna(1)
)
df['Salles de bains'] = df['Salles de bains'].round()
df['Salles de bains'] = df['Salles de bains'].replace(0, np.nan)
df['Salles de bains'] = df.groupby('Type de bien')['Salles de bains'].transform(lambda x: x.fillna(x.median()))

df.dropna(subset=['Type de bien', 'Localisation'], inplace=True)

# Filter 'Prix' to handle unrealistic values
df.loc[(df['Prix'] < 50000) | (df['Prix'] > 1300000), 'Prix'] = 0

print("Missing values after cleaning:")
print(df[['Superficie', 'Chambres', 'Salles de bains', 'Type de bien', 'Localisation']].isnull().sum())


Missing values after cleaning:
Superficie         0
Chambres           0
Salles de bains    0
Type de bien       0
Localisation       0
dtype: int64


In [13]:
# Quantile values for Superficie
superficie_quantiles = df['Superficie'].quantile([0.022, 0.53, 0.98])
print("Superficie Quantiles:")
print(superficie_quantiles)

# Quantile values for Nb_Chambres
nb_chambres_quantiles = df['Chambres'].quantile([0.0013, 0.5, 0.98])
print("Chambres Quantiles:")
print(nb_chambres_quantiles)

# Quantile values for Nb_Salles De bain
nb_salles_de_bain_quantiles = df['Salles de bains'].quantile([0.0013, 0.5, 0.98])
print("Salles de bains Quantiles:")
print(nb_salles_de_bain_quantiles)

Superficie Quantiles:
0.022      70.00
0.530     350.00
0.980    1480.28
Name: Superficie, dtype: float64
Chambres Quantiles:
0.0013     1.0
0.5000     4.0
0.9800    10.0
Name: Chambres, dtype: float64
Salles de bains Quantiles:
0.0013    1.0
0.5000    2.0
0.9800    6.0
Name: Salles de bains, dtype: float64


In [17]:
df = df[
    (df['Superficie'] >= superficie_quantiles[0.022]) & 
    (df['Superficie'] <= superficie_quantiles[0.98]) & 
    (df['Chambres'] >= nb_chambres_quantiles[0.0013]) & 
    (df['Chambres'] <= nb_chambres_quantiles[0.98]) & 
    (df['Salles de bains'] >= nb_salles_de_bain_quantiles[0.0013]) & 
    (df['Salles de bains'] <= nb_salles_de_bain_quantiles[0.98])
]

In [18]:
df.to_csv('data/final_data.csv', index=False)